In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import unicodedata

### WEB SCRAPING BERLIN

In [2]:
url = "https://en.wikipedia.org/wiki/Berlin"

In [3]:
response = requests.get(url)

In [4]:
soup = BeautifulSoup(response.content, "html.parser")

soup.prettify

<bound method Tag.prettify of <!DOCTYPE html>

<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>Berlin - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":false,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"c6f41c17-ff1e-490c-b84b-317a0aca865f","wgCSPNonce":false,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"Berlin","wgTitle":"Berlin","wgCurRevisionId":1082324147,"wgRevisionId":1082324147,"wgArticleId":3354,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["CS1 German-language sources (de)","CS1 maint: multiple names: authors list","CS1 maint: bot: original URL status unknown","Webarchive template way

In [5]:
country = soup.select("#mw-content-text > div.mw-parser-output > table.infobox.ib-settlement.vcard > tbody > tr:nth-child(9) > td > a")[0].get_text()
country

'Germany'

In [6]:
state = soup.select("#mw-content-text > div.mw-parser-output > table.infobox.ib-settlement.vcard > tbody > tr:nth-child(10) > td > a")[0].get_text()
state

'Berlin'

In [7]:
mayor = soup.select("#mw-content-text > div.mw-parser-output > table.infobox.ib-settlement.vcard > tbody > tr:nth-child(13) > td > a:nth-child(1)")[0].get_text()
mayor

'Franziska Giffey'

In [8]:
area = soup.select("#mw-content-text > div.mw-parser-output > table.infobox.ib-settlement.vcard > tbody > tr:nth-child(15) > td")[0].get_text()
area

'891.7\xa0km2 (344.3\xa0sq\xa0mi)'

In [9]:
population = soup.select("#mw-content-text > div.mw-parser-output > table.infobox.ib-settlement.vcard > tbody > tr:nth-child(18) > td")[0].get_text()
population

'3,769,495'

In [10]:
soup.h1.text == soup.h1.get_text() == soup.select("h1")[0].get_text()


True

In [11]:
title = soup.find(id="firstHeading").get_text()
title

'Berlin'

In [6]:
soup.select("td.infobox-data")[0].get_text()

berlin = {"Country":[], "City":[],"Mayor":[],"Area":[],"Elevation":[],"Population_City":[],"Population_Urban":[],"Population_Metro":[]}

berlin["Country"].append(soup.select("td.infobox-data")[0].get_text())
berlin["City"].append(soup.select("td.infobox-data")[1].get_text())
berlin["Mayor"].append(soup.select("td.infobox-data")[3].get_text())
berlin["Area"].append(soup.select("td.infobox-data")[4].get_text())
berlin["Elevation"].append(soup.select("td.infobox-data")[5].get_text())
berlin["Population_City"].append(soup.select("td.infobox-data")[6].get_text())
berlin["Population_Urban"].append(soup.select("td.infobox-data")[7].get_text())
berlin["Population_Metro"].append(soup.select("td.infobox-data")[8].get_text())

berlin_df = pd.DataFrame(berlin)
berlin_df

,Country,City,Mayor,Area,Elevation,Population_City,Population_Urban,Population_Metro
0,Germany,Berlin,Franziska Giffey (SPD),891.7 km2 (344.3 sq mi),34 m (112 ft),"3,769,495","4,473,101","6,144,600"


In [7]:
berlin_df.dtypes

Country             object
City                object
Mayor               object
Area                object
Elevation           object
Population_City     object
Population_Urban    object
Population_Metro    object
dtype: object

## GENERAL

When we have a Python Library, we can access the API and the information using that Library, which is easier.
If we do NOT have a specific Library for that API, we follow a standard process:

1. Make a request to the API
2. Get a JSON file
3. Navigate through the JSON file and fetch some data

### GET WEATHER DATA

In [6]:
# !pip install pyowm
from pyowm import OWM
from pyowm.utils import config
from pyowm.utils import timestamps
owm = OWM('YOUR KEY')

In [7]:
OWM_key = 'YOUR KEY'
city = "Berlin"
country = "DE"

mgr = owm.weather_manager()

response = requests.get(f'http://api.openweathermap.org/data/2.5/forecast/?q={city},{country}&appid={OWM_key}&units=metric&lang=en')

weather_json = pd.json_normalize(response.json()['list'])

weather_df = pd.DataFrame(weather_json).filter(['dt', 'main.temp','clouds.all', 'wind.speed', 'rain.3h'])

weather_df['dt'] = pd.to_datetime(weather_df['dt'], unit='s')

weather_df['Where'] = city + ', ' + country

weather_df

,dt,main.temp,clouds.all,wind.speed,rain.3h,Where
0,2022-04-07 15:00:00,9.54,75,8.80,1.91,"Berlin, DE"
1,2022-04-07 18:00:00,9.50,78,8.77,0.14,"Berlin, DE"
2,2022-04-07 21:00:00,8.69,92,9.51,0.37,"Berlin, DE"
3,2022-04-08 00:00:00,7.09,99,10.60,0.70,"Berlin, DE"
4,2022-04-08 03:00:00,5.40,100,9.72,0.61,"Berlin, DE"
5,2022-04-08 06:00:00,5.05,97,7.25,0.18,"Berlin, DE"
6,2022-04-08 09:00:00,7.11,99,7.71,0.20,"Berlin, DE"
7,2022-04-08 12:00:00,8.78,96,7.02,NaN,"Berlin, DE"
8,2022-04-08 15:00:00,8.60,100,6.23,0.39,"Berlin, DE"
9,2022-04-08 18:00:00,6.94,100,4.71,0.37,"Berlin, DE"


### GET CITIES DATA

In [8]:
# cities = ['Berlin', 'Hamburg', 'Frankfurt','Munich','Stuttgart','Leipzig','Cologne','Dresden','Hannover','Paris', 'Barcelona','Lisbon','Madrid']
cities = ['Berlin','Paris','Amsterdam','Barcelona','Rome','Lisbon','Prague','Vienna','Madrid']

def City_info(soup):
    
    ret_dict = {}
    ret_dict['city'] = soup.h1.get_text()
    
    
    if soup.select_one('.mergedrow:-soup-contains("Mayor")>.infobox-label') != None:
        i = soup.select_one('.mergedrow:-soup-contains("Mayor")>.infobox-label')
        mayor_name_html = i.find_next_sibling()
        mayor_name = unicodedata.normalize('NFKD',mayor_name_html.get_text())
        ret_dict['mayor']  = mayor_name
    
    if soup.select_one('.mergedrow:-soup-contains("City")>.infobox-label') != None:
        j =  soup.select_one('.mergedrow:-soup-contains("City")>.infobox-label')
        area = j.find_next_sibling('td').get_text()
        ret_dict['city_size'] = unicodedata.normalize('NFKD',area)

    if soup.select_one('.mergedtoprow:-soup-contains("Elevation")>.infobox-data') != None:
        k = soup.select_one('.mergedtoprow:-soup-contains("Elevation")>.infobox-data')
        elevation_html = k.get_text()
        ret_dict['elevation'] = unicodedata.normalize('NFKD',elevation_html)
    
    if soup.select_one('.mergedtoprow:-soup-contains("Population")') != None:
        l = soup.select_one('.mergedtoprow:-soup-contains("Population")')
        c_pop = l.findNext('td').get_text()
        ret_dict['city_population'] = c_pop
    
    if soup.select_one('.infobox-label>[title^=Urban]') != None:
        m = soup.select_one('.infobox-label>[title^=Urban]')
        u_pop = m.findNext('td')
        ret_dict['urban_population'] = u_pop.get_text()

    if soup.select_one('.infobox-label>[title^=Metro]') != None:
        n = soup.select_one('.infobox-label>[title^=Metro]')
        m_pop = n.findNext('td')
        ret_dict['metro_population'] = m_pop.get_text()
    
    if soup.select_one('.latitude') != None:
        o = soup.select_one('.latitude')
        ret_dict['lat'] = o.get_text()

    if soup.select_one('.longitude') != None:    
        p = soup.select_one('.longitude')
        ret_dict['long'] = p.get_text()
    
    return ret_dict



list_of_city_info = []

for city in cities:
    url = 'https://en.wikipedia.org/wiki/{}'.format(city)
    web = requests.get(url,'html.parser')
    soup = BeautifulSoup(web.content)
    list_of_city_info.append(City_info(soup))

df_cities = pd.DataFrame(list_of_city_info)

df_cities

,city,mayor,city_size,elevation,city_population,urban_population,metro_population,lat,long
0,Berlin,Franziska Giffey (SPD),891.7 km2 (344.3 sq mi),34 m (112 ft),"3,769,495","4,473,101","6,144,600",52°31′12″N,13°24′18″E
1,Paris,Anne Hidalgo (PS),NaN,28–131 m (92–430 ft) (avg. 78 m or 256 ft),"2,165,423","10,785,092","13,024,518",48°51′24″N,2°21′08″E
2,Amsterdam,Femke Halsema (GL),NaN,−2 m (−7 ft),"905,234","1,558,755",NaN,52°22′N,4°54′E
3,Barcelona,Ada Colau Ballano[1] (Barcelona en Comú),101.4 km2 (39.2 sq mi),12 m (39 ft),"1,620,343","4,840,000[3]","5,474,482[4]",41°23′N,2°11′E
4,Rome,Strong Mayor–Council,"4,342,212[2]",21 m (69 ft),1st in Italy (3rd in the EU),NaN,Rome Capital,41°53′36″N,12°28′58″E
5,Lisbon,Carlos Moedas,NaN,2 m (7 ft),"544,851[1]","2,719,000[4]","2,871,133[2][3]",38°43′31″N,9°09′00″W
6,Prague,Zdeněk Hřib (Pirates),NaN,NaN,"1,335,084",NaN,"2,709,418[4]",50°5′N,14°25′E
7,Vienna,Michael Ludwig (SPÖ),NaN,"151 (Lobau) – 542 (Hermannskogel) m (495–1,778...",1st in Austria (6th in EU),"1,911,191 (01−01−20)","2,600,000",48°12′N,16°22′E
8,Madrid,José Luis Martínez-Almeida (PP),NaN,"650 m (2,130 ft)","3,223,334","6,345,000 (2,019)[3]","6,791,667 (2,018)[2]",40°25′N,3°43′W


### GET FLIGHTS DATA

In [9]:
API_key = "YOUR KEY"

airport_icoa = "EDDB" #Berlin airport code

# times, between which, we would like to receive flight information
to_local_time = "2022-04-06T00:00"
from_local_time = "2022-04-06T12:00"

url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{airport_icoa}/{to_local_time}/{from_local_time}"

querystring = {"withLeg":"true","withCancelled":"true","withCodeshared":"true","withCargo":"true","withPrivate":"false","withLocation":"false"}

headers = {
	"X-RapidAPI-Host": "aerodatabox.p.rapidapi.com",
	"X-RapidAPI-Key": API_key
}

response = requests.request("GET", url, headers=headers, params=querystring)

### Making a dataframe of flight data using json_normalize
### By far the easiest method, and you don't need to worry about missing data messing up the lengths of your lists

# Step 1: make a dataframe from all of the information in the json
arrivals = pd.json_normalize(response.json()['arrivals'])

# Step2: select only the columns you want to incude on your database
flights_df = pd.DataFrame(arrivals
    .filter(['departure.airport.name','arrival.scheduledTimeLocal',
             'arrival.terminal','status','aircraft.model'])
    .assign(icao_code = airport_icoa))

flights_df

# from IPython.display import JSON
# JSON(response.json())

,departure.airport.name,arrival.scheduledTimeLocal,arrival.terminal,status,aircraft.model,icao_code
0,Stuttgart,2022-04-06 01:16+02:00,NaN,Arrived,Airbus A320-100/200,EDDB
1,Cologne,2022-04-06 05:36+02:00,NaN,Arrived,Boeing 767,EDDB
2,Cologne,2022-04-06 07:25+02:00,1,Arrived,Boeing 737,EDDB
3,Duesseldorf,2022-04-06 07:35+02:00,1,Unknown,Airbus A319,EDDB
4,Munich,2022-04-06 07:35+02:00,1,Arrived,Airbus A319,EDDB
...,...,...,...,...,...,...
86,London,2022-04-06 11:46+02:00,NaN,Arrived,Embraer EMB 190,EDDB
87,Istanbul,2022-04-06 11:20+02:00,1,Arrived,Boeing 737-800,EDDB
88,Unknown,2022-04-06 11:42+02:00,NaN,Arrived,Airbus A319,EDDB
89,Vienna,2022-04-06 11:36+02:00,NaN,Arrived,Airbus A320-100/200,EDDB


In [6]:
# alternative code for the Flights

# import pandas as pd
# import datetime
# import requests
# def get_flight_data(icao):
#     today = datetime.date.today()
#     tomorrow = str(today + datetime.timedelta(days=1))
    
#     hours = [["00:00","11:59"],["12:00","23:59"]]
#     flight_data = []
#     for time in hours:
#         url =      f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{tomorrow}T{time[0]}/{tomorrow}T{time[1]}"
#         querystring = {"withLeg":"true","direction":"Arrival","withCancelled":"false","withCodeshared":"true","withCargo":"false","withPrivate":"false","withLocation":"false"}
#         headers = {
#             'x-rapidapi-host': "aerodatabox.p.rapidapi.com",
#             'x-rapidapi-key': flights_key
#             }
#         flight_data.append(requests.request("GET", url,   headers=headers, params=querystring).json())
#     return flight_data

### GET AIRPORTS' DATA

In [10]:
airports_cities = (
pd.read_csv('airports.csv')
    .query('type == "large_airport"')
    .filter(['name','latitude_deg','longitude_deg','iso_country','iso_region','municipality','gps_code','iata_code'])
    .rename(columns={'gps_code':'icao_code'})
    .assign(municipality_iso_country = lambda x: x['municipality'] + ',' + x['iso_country'])
)
airports_cities.head()

,name,latitude_deg,longitude_deg,iso_country,iso_region,municipality,icao_code,iata_code,municipality_iso_country
10890,Honiara International Airport,-9.428000,160.054993,SB,SB-CT,Honiara,AGGH,HIR,"Honiara,SB"
12461,Port Moresby Jacksons International Airport,-9.443380,147.220001,PG,PG-NCD,Port Moresby,AYPY,POM,"Port Moresby,PG"
12981,Keflavik International Airport,63.985001,-22.605600,IS,IS-2,Reykjavík,BIKF,KEF,"Reykjavík,IS"
13028,Priština Adem Jashari International Airport,42.572800,21.035801,XK,XK-01,Prishtina,BKPR,PRN,"Prishtina,XK"
17254,Guodu Air Base,36.001741,117.632010,CN,CN-37,"Xintai, Tai'an",NaN,NaN,"Xintai, Tai'an,CN"


In [19]:
airports_cities.query('municipality == "Berlin"')

,name,latitude_deg,longitude_deg,iso_country,iso_region,municipality,icao_code,iata_code,municipality_iso_country
20244,Berlin Brandenburg Airport,52.351389,13.493889,DE,DE-BR,Berlin,EDDB,BER,"Berlin,DE"


### CHECK THE TABLES AND THE DATA TYPES BEFORE SENDING IT TO SQL (note: very important!!!)

Step 1: Check your datatypes are correct

It would be nice if the date time was in a datetime format


In [22]:
flights_df['arrival.scheduledTimeLocal'] = pd.to_datetime(flights_df['arrival.scheduledTimeLocal'])
flights_df.dtypes

KeyError: 'arrival.scheduledTimeLocal'

Drop the duplicates

In [14]:
cities = airports_cities.filter(['municipality','iso_country','municipality_iso_country']).drop_duplicates()
cities.head()

,municipality,iso_country,municipality_iso_country
10890,Honiara,SB,"Honiara,SB"
12461,Port Moresby,PG,"Port Moresby,PG"
12981,Reykjavík,IS,"Reykjavík,IS"
13028,Prishtina,XK,"Prishtina,XK"
17254,"Xintai, Tai'an",CN,"Xintai, Tai'an,CN"


# Summing up what we have:

In [13]:
weather_df.rename(columns={"dt": "date_time", "main.temp": "temperature", "clouds.all": "clouds", "wind.speed": "wind", "rain.3h":"rain","Where":"municipality_iso_country"}, inplace=True)
weather_df.head()

,date_time,temperature,clouds,wind,rain,municipality_iso_country
0,2022-04-07 15:00:00,9.54,75,8.80,1.91,"Berlin, DE"
1,2022-04-07 18:00:00,9.50,78,8.77,0.14,"Berlin, DE"
2,2022-04-07 21:00:00,8.69,92,9.51,0.37,"Berlin, DE"
3,2022-04-08 00:00:00,7.09,99,10.60,0.70,"Berlin, DE"
4,2022-04-08 03:00:00,5.40,100,9.72,0.61,"Berlin, DE"


In [17]:
weather_df.dtypes

date_time                   datetime64[ns]
temperature                        float64
clouds                               int64
wind                               float64
rain                               float64
municipality_iso_country            object
dtype: object

In [15]:
df_cities["municipality_iso_country"] = ["Berlin, DE", "Paris, FR", "Amsterdam, NL", "Barcelona, ES", "Rome, IT", "Lisbon, PT", "Prague, CZE", "Vienna, AT", "Madrid, ES"]

In [16]:
df_cities

,city,mayor,city_size,elevation,city_population,urban_population,metro_population,lat,long,municipality_iso_country
0,Berlin,Franziska Giffey (SPD),891.7 km2 (344.3 sq mi),34 m (112 ft),"3,769,495","4,473,101","6,144,600",52°31′12″N,13°24′18″E,"Berlin, DE"
1,Paris,Anne Hidalgo (PS),NaN,28–131 m (92–430 ft) (avg. 78 m or 256 ft),"2,165,423","10,785,092","13,024,518",48°51′24″N,2°21′08″E,"Paris, FR"
2,Amsterdam,Femke Halsema (GL),NaN,−2 m (−7 ft),"905,234","1,558,755",NaN,52°22′N,4°54′E,"Amsterdam, NL"
3,Barcelona,Ada Colau Ballano[1] (Barcelona en Comú),101.4 km2 (39.2 sq mi),12 m (39 ft),"1,620,343","4,840,000[3]","5,474,482[4]",41°23′N,2°11′E,"Barcelona, ES"
4,Rome,Strong Mayor–Council,"4,342,212[2]",21 m (69 ft),1st in Italy (3rd in the EU),NaN,Rome Capital,41°53′36″N,12°28′58″E,"Rome, IT"
5,Lisbon,Carlos Moedas,NaN,2 m (7 ft),"544,851[1]","2,719,000[4]","2,871,133[2][3]",38°43′31″N,9°09′00″W,"Lisbon, PT"
6,Prague,Zdeněk Hřib (Pirates),NaN,NaN,"1,335,084",NaN,"2,709,418[4]",50°5′N,14°25′E,"Prague, CZE"
7,Vienna,Michael Ludwig (SPÖ),NaN,"151 (Lobau) – 542 (Hermannskogel) m (495–1,778...",1st in Austria (6th in EU),"1,911,191 (01−01−20)","2,600,000",48°12′N,16°22′E,"Vienna, AT"
8,Madrid,José Luis Martínez-Almeida (PP),NaN,"650 m (2,130 ft)","3,223,334","6,345,000 (2,019)[3]","6,791,667 (2,018)[2]",40°25′N,3°43′W,"Madrid, ES"


In [18]:
df_cities.dtypes

city                        object
mayor                       object
city_size                   object
elevation                   object
city_population             object
urban_population            object
metro_population            object
lat                         object
long                        object
municipality_iso_country    object
dtype: object

In [19]:
flights_df.rename(columns={"departure.airport.name": "dep_airport", "arrival.scheduledTimeLocal": "sched_arriv_local_time", "arrival.terminal": "terminal", "aircraft.model": "aircraft"}, inplace=True)

In [24]:
flights_df.head()

,dep_airport,sched_arriv_local_time,terminal,status,aircraft,icao_code
0,Stuttgart,2022-04-06 01:16:00+02:00,NaN,Arrived,Airbus A320-100/200,EDDB
1,Cologne,2022-04-06 05:36:00+02:00,NaN,Arrived,Boeing 767,EDDB
2,Cologne,2022-04-06 07:25:00+02:00,1,Arrived,Boeing 737,EDDB
3,Duesseldorf,2022-04-06 07:35:00+02:00,1,Unknown,Airbus A319,EDDB
4,Munich,2022-04-06 07:35:00+02:00,1,Arrived,Airbus A319,EDDB


In [26]:
airports_cities.head()

,name,latitude_deg,longitude_deg,iso_country,iso_region,municipality,icao_code,iata_code,municipality_iso_country
10890,Honiara International Airport,-9.428000,160.054993,SB,SB-CT,Honiara,AGGH,HIR,"Honiara,SB"
12461,Port Moresby Jacksons International Airport,-9.443380,147.220001,PG,PG-NCD,Port Moresby,AYPY,POM,"Port Moresby,PG"
12981,Keflavik International Airport,63.985001,-22.605600,IS,IS-2,Reykjavík,BIKF,KEF,"Reykjavík,IS"
13028,Priština Adem Jashari International Airport,42.572800,21.035801,XK,XK-01,Prishtina,BKPR,PRN,"Prishtina,XK"
17254,Guodu Air Base,36.001741,117.632010,CN,CN-37,"Xintai, Tai'an",NaN,NaN,"Xintai, Tai'an,CN"


In [27]:
airports_cities.dtypes

name                         object
latitude_deg                float64
longitude_deg               float64
iso_country                  object
iso_region                   object
municipality                 object
icao_code                    object
iata_code                    object
municipality_iso_country     object
dtype: object

# Connect the Dataframes to MySQL

SQLAlchemy is the simplest way to connect Python to any SQL. After installing it with ``pip install SQLAlchemy`` and ``importing`` it into your notebook, you will have to define the details to ``connect to your database``. Then, the method ``pandas.DataFrame.to_sql()`` will do the rest for you, converting a DataFrame into a MySQL table in a single step. If the table does not exist yet in your database, it will be created, and you will not even have to worry about data types —they will be inferred from the existing data types in the DataFrame. You can always change the table from MySQLWorkbench using ALTER TABLE together with the MODIFY clause.

Step1: Install connections packages and import them into the notebook

In [28]:
# !pip install sqlalchemy
# !pip install PyMySQL

import sqlalchemy
import pymysql

Step2: set connection variables. Make sure your schema exists on MySQLWorkbench.

If it doesn't already exist. In the schema box, right click in some white space. Select Create Scehema. Create a Schema with the same name as the schema variable in the cell below

In [77]:
# schema="gans"
# host="127.0.0.1"
# user="root"
# password="YOUR PASSWORD"
# port=3306
# con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

Step3: send data to MySQLWorkbench (ONLY ONCE! or it will append the same data over and over again)

In [79]:
# flights_df.to_sql('flights', con=con, if_exists='append', index=False)

# flights_df is the name of the dataframe I wish to send
# .to_sql is the function that sends our data to SQL
# flights is the name of the table we would like to create in our schema
# con is our connection information that we typed out above
# 'append' means that we add the information we're sending to any existing information (as oppossed to writing over it)
# index=False means that we do not create a new index in SQL for this table

In [80]:
# df_cities.dropna().to_sql('cities', con=con, if_exists='append', index=False)

In [81]:
# weather_df.dropna().to_sql('weather', con=con, if_exists='append', index=False)

In [82]:
# airports_cities.dropna().to_sql('airports_cities', con=con, if_exists='append', index=False)

Other

In [ ]:
# weather_df.assign(datetime = lambda x: pd.to_datetime(x['datetime'])).to_sql('weather', if_exists='append', con=con, index=False)

In [ ]:
# import numpy as np
# (
# arrivals_berlin
#     .replace({np.nan},'unknown')
#     .assign(sched_arr_loc_time = lambda x: pd.to_datetime(x['sched_arr_loc_time']))
#     .to_sql('arrivals', if_exists='append', con=con, index=False))

#### Run sql queries in our python session

In [29]:
# pd.read_sql(
#     sql = """
#         select * from flights
#         where status = "Unknown"
#     """,
#     con = con
# )

Or add the Primary and Foreign Keys to our tables:

In [1]:
# con = "YOUR CONNECTION STRING TO DB"
#    engine = sqlalchemy.create_engine(con)
#    with engine.connect() as engine:
#         # Add primary key
#         engine.execute('''
#         ALTER TABLE airports 
#         ADD PRIMARY KEY (airport_id);
#         '''
#         )
#         # Add foreign key
#         engine.execute('''
#         ALTER TABLE airports 
#         ADD FOREIGN KEY (municipality_country) REFERENCES cities(municipality_country);
#         ''')

### NOTE ON MYSQL: 

We can apply modifications to the datatypes and the relations between different tables (Primary, Foreign keys) directly in the Schema of the Database, by clicking to `DATABASE > Sync Model`

### Export the tables also into the ``AWS<>MySQL instance`` (also called Connection) after creating there a database (schema) called Gans

In [30]:
schema="gans"
host="wbs-projectcloud-db.cy0cju6fkpzt.us-east-1.rds.amazonaws.com"
user="admin"
password="YOUR PASSWORD"
port=3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [31]:
flights_df.to_sql('flights', con=con, if_exists='append', index=False)
df_cities.dropna().to_sql('cities', con=con, if_exists='append', index=False)
weather_df.dropna().to_sql('weather', con=con, if_exists='append', index=False)
airports_cities.dropna().to_sql('airports_cities', con=con, if_exists='append', index=False)

# RECAP FOR LAMBDA

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import unicodedata

### Weather API

In [ ]:
from pyowm import OWM
from pyowm.utils import config
from pyowm.utils import timestamps
owm = OWM('YOUR KEY')

In [ ]:
OWM_key = 'YOUR KEY'
city = "Berlin"
country = "DE"

mgr = owm.weather_manager()

response = requests.get(f'http://api.openweathermap.org/data/2.5/forecast/?q={city},{country}&appid={OWM_key}&units=metric&lang=en')

weather_json = pd.json_normalize(response.json()['list'])

weather_df = pd.DataFrame(weather_json).filter(['dt', 'main.temp','clouds.all', 'wind.speed', 'rain.3h'])

weather_df['dt'] = pd.to_datetime(weather_df['dt'], unit='s')

weather_df['Where'] = city + ', ' + country

weather_df

In [ ]:
weather_df.rename(columns={"dt": "date_time", "main.temp": "temperature", "clouds.all": "clouds", "wind.speed": "wind", "rain.3h":"rain","Where":"municipality_iso_country"}, inplace=True)
weather_df.head()

### Flights API

In [ ]:
API_key = "YOUR KEY"

airport_icoa = "EDDB" #Berlin airport code

# times, between which, we would like to receive flight information
to_local_time = "2022-04-06T00:00"
from_local_time = "2022-04-06T12:00"

url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{airport_icoa}/{to_local_time}/{from_local_time}"

querystring = {"withLeg":"true","withCancelled":"true","withCodeshared":"true","withCargo":"true","withPrivate":"false","withLocation":"false"}

headers = {
	"X-RapidAPI-Host": "aerodatabox.p.rapidapi.com",
	"X-RapidAPI-Key": API_key
}

response = requests.request("GET", url, headers=headers, params=querystring)

### Making a dataframe of flight data using json_normalize
### By far the easiest method, and you don't need to worry about missing data messing up the lengths of your lists

# Step 1: make a dataframe from all of the information in the json
arrivals = pd.json_normalize(response.json()['arrivals'])

# Step2: select only the columns you want to incude on your database
flights_df = pd.DataFrame(arrivals
    .filter(['departure.airport.name','arrival.scheduledTimeLocal',
             'arrival.terminal','status','aircraft.model'])
    .assign(icao_code = airport_icoa))

flights_df

In [ ]:
flights_df['arrival.scheduledTimeLocal'] = pd.to_datetime(flights_df['arrival.scheduledTimeLocal'])
flights_df.dtypes

In [ ]:
flights_df.rename(columns={"departure.airport.name": "dep_airport", "arrival.scheduledTimeLocal": "sched_arriv_local_time", "arrival.terminal": "terminal", "aircraft.model": "aircraft"}, inplace=True)
flights_df.head()


### Connect to MySQL

In [ ]:
import sqlalchemy
import pymysql

In [ ]:
schema="gans"
host="wbs-projectcloud-db.cy0cju6fkpzt.us-east-1.rds.amazonaws.com"
user="admin"
password="YOUR PASSWORD"
port=3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [ ]:
flights_df.to_sql('flights', con=con, if_exists='append', index=False)
df_cities.dropna().to_sql('cities', con=con, if_exists='append', index=False)
weather_df.dropna().to_sql('weather', con=con, if_exists='append', index=False)
airports_cities.dropna().to_sql('airports_cities', con=con, if_exists='append', index=False)

In [2]:
# PyMySQL
# arn:aws:lambda:us-east-1:770693421928:layer:Klayers-python38-PyMySQL:4

In [ ]:
# SQLAlchemy
# arn:aws:lambda:us-east-1:770693421928:layer:Klayers-p38-SQLAlchemy:1